In [4]:
import pickle
import nltk as nl
import sklearn as mp
from sklearn.linear_model import LogisticRegression, LinearRegression
import requests as rq
import sqlalchemy as sqla
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn import metrics
import matplotlib as mpl
import datetime as dt
import numpy as np
from sktime.datasets import load_airline
dpi, w, h = 160, 12, 8
mpl.rcParams.update({'figure.figsize': (w, h)})
mpl.rcParams.update({'figure.dpi': dpi})
import json
# import seaborn as sns
# Allows plots to appear directly in the notebook.
%matplotlib inline
%matplotlib inline
import os

In [7]:
os.environ['PYTHONXEXC'] = 'off'
print(os.environ['PYTHONXEXC'])

off


In [6]:
%env PYTHONXEXC=off

env: PYTHONXEXC=off


In [ ]:
%%capture
# capture magic method allows to hide the output cell
# define tables here so that they would be visible during the model prediciton creation
CurrentWeather = """
    CREATE TABLE IF NOT EXISTS CurrentWeather (
        time_update DATETIME NOT NULL,
        current_weather_id INT AUTO_INCREMENT PRIMARY KEY,
        feels_like DECIMAL(5, 2) NOT NULL,
        temperature_min DECIMAL(5, 2) NOT NULL,
        temperature_max DECIMAL(5, 2) NOT NULL,
        weather_description VARCHAR(120),
        wind_speed DECIMAL(5, 2) NOT NULL,
        wind_gust DECIMAL(5, 2) NOT NULL DEFAULT 0
    );
"""
# Copying the extreme weather table here, to merge timestamp later
ExtremeWeather = """
    CREATE TABLE IF NOT EXISTS ExtremeWeather (
        time_update DATETIME NOT NULL,
        extreme_weather_id INT AUTO_INCREMENT PRIMARY KEY,
        temp_min DECIMAL(5, 2) NOT NULL,
        temp_max DECIMAL(5, 2) NOT NULL,
        wind_speed DECIMAL(5, 2) NOT NULL,
        gust_speed DECIMAL(5, 2) NOT NULL DEFAULT 0,
        rain_3h DECIMAL(5, 2) NOT NULL DEFAULT 0
    );
"""

FiveDayPrediction = """
    CREATE TABLE IF NOT EXISTS FiveDayPrediction (
        time_update DATETIME NOT NULL,
        forecast_id INT AUTO_INCREMENT PRIMARY KEY,
        temp_min DECIMAL(5, 2) NOT NULL,
        temp_max DECIMAL(5, 2) NOT NULL,
        wind_speed DECIMAL(5, 2) NOT NULL,
        gust DECIMAL(5, 2) NOT NULL DEFAULT 0,
        rain_3h DECIMAL(5, 2) NOT NULL DEFAULT 0
    );
"""
# static station data
Station = """
    CREATE TABLE IF NOT EXISTS station (
        number INT NOT NULL,
        name VARCHAR (120),
        address VARCHAR(256),
        banking TINYINT(1), 
        bonus TINYINT(1),
        position_lat REAL,
        position_lng REAL, 
        PRIMARY KEY (number)
    );
"""
# Dynamic station data
StationStatus = """
    DROP TABLE IF EXISTS station_status;
    CREATE TABLE station_status (
        station_number INT NOT NULL,
        status VARCHAR(256) NOT NULL,
        last_update DATETIME,
        empty_stands_number INT,
        total_bikes INT,
        mechanical_bikes INT,
        electrical_internal_battery_bikes INT,
        electrical_removable_battery_bikes INT,
        PRIMARY KEY (station_number, last_update),
        FOREIGN KEY (station_number) REFERENCES station(number)
        ON DELETE CASCADE
    );
"""

In [ ]:
try:
    # Read database connection details from config.json
    with open('config.json', 'r') as f:
        config = json.load(f)
    connection_url = config.get('url')
    if not connection_url:
        raise ValueError("Connection URL is missing in the configuration file.")
except FileNotFoundError:
    print("Error: config.json file not found.")
    exit()
except json.JSONDecodeError:
    print("Error: Unable to decode config.json file. Make sure it's in proper JSON format.")
    exit()
except KeyError:
    print("Error: 'url' key is missing in the config.json file.")
    exit()
except ValueError as e:
    print(f"Error: {e}")
    exit()

try:
    # Attempt to establish connection to the database
    engine = create_engine(connection_url)
    connection = engine.connect()
    print("Connection established successfully.")
except Exception as e:
    print("An unexpected error occurred while establishing the connection:", e)
    exit()

connection.close()

<h2><center>Merging Timestamps</center></p>

In [ ]:
df_station = pd.read_sql('station', engine)
df_station = pd.read_sql('station_status', engine)
df_weather = pd.read_sql('CurrentWeather', engine)
df_weather = pd.read_sql('FiveDayPrediction', engine)
df_weather = pd.read_sql('ExtremeWeather', engine)

# tables = ['CurrentWeather', 'ExtremeWeather', 'FiveDayPrediction', 'Station', 'StationStatus']

In [7]:
%%capture
# Histograms for numerical variables
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    plt.figure(figsize=(8, 4))
    sns.histplot(df[col], bins=20, kde=True)
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

# Bar plot for categorical variables
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    plt.figure(figsize=(8, 4))
    df[col].value_counts().plot(kind='bar')
    plt.title(f'Bar plot of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.show()

# Data Relationships
# Correlation matrix
correlation_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

# Scatter plots for relationships
for col in numerical_cols:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=col, y=df.columns[-1], data=df)
    plt.title(f'Scatter plot of {col} vs. {df.columns[-1]}')
    plt.xlabel(col)
    plt.ylabel(df.columns[-1])
    plt.show()

# Outliers and Anomalies
# Box plots for outlier detection
for col in numerical_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df[col])
    plt.title(f'Box plot of {col}')
    plt.xlabel(col)
    plt.show()

NameError: name 'df' is not defined

In [5]:
# Data Inspection
# Display the first 5 rows of the DataFrame
df.head()
# Display the shape of the DataFrame (number of rows and columns)
df.shape
# Display information about the DataFrame, including data types and memory usage
df.info()
# Count the number of duplicate rows in the DataFrame
df.duplicated().sum()
# Descriptive Statistics
df.describe()
# types of the data 
df.dtypes

NameError: name 'df' is not defined

In [14]:
# the code below is based on code presented in module COMP47350 

In [15]:
def check_data_quality(df):
    """
    Check the data quality of a DataFrame for missing, unusual, and unknown values.
    
    Parameters:
    - df: pandas DataFrame
    
    Returns:
    - Dictionary containing information about missing, unusual, and unknown values.
    """
    quality_report = {}
    
    # Check for missing values
    missing_values = df.isnull().sum()
    quality_report['missing_values'] = missing_values[missing_values > 0]
    
    # Check for unusual values
    unusual_values = {}
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numerical_cols:
        unusual_values[col] = df[col].describe()
    
    quality_report['unusual_values'] = unusual_values
    
    # Check for unknown values (if applicable)
    # You need to define what constitutes an "unknown" value in your context
    
    return quality_report

# Usage example
quality_report = check_data_quality(df)
print("Missing Values:")
print(quality_report['missing_values'])

print("\nUnusual Values:")
for col, stats in quality_report['unusual_values'].items():
    print(f"\n{col}:")
    print(stats)

NameError: name 'df' is not defined

In [16]:
# Check the shape of the DataFrame
num_rows, num_cols = df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)

NameError: name 'df' is not defined

In [18]:
df.head(10)

NameError: name 'df' is not defined

In [20]:
df.shape

NameError: name 'df' is not defined

In [21]:
# Convert data types to relevant data types
categorical_columns = df[['time_update', 'temp_min', 'temp_max','wind_speed', 'gust_speed', 'rain_3h']].columns
# Convert data type to category for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')  

continuous_columns = df.select_dtypes(['int64']).columns
datetime_columns = df.select_dtypes(['datetime64[ns]']).columns
df.dtypes

NameError: name 'df' is not defined

In [22]:
# Var_Corr = df.corr()
# plt.subplots(figsize=(50, 50))
# plot the heatmap and annotation on it
#sns.heatmap(Var_Corr, xticklabels=Var_Corr.columns, yticklabels=Var_Corr.columns, annot=True)

username = api_keys.DB_USER
password = api_keys.DB_PASS
url = api_keys.DB_URL

engine = create_engine("mysql+pymysql://{0}:{1}@{2}".format(DB_USER, DB_PASS, DB_URL), echo=True) 
connection = engine.connect()

statement = """SELECT dayname(availability.time_queried) as dayquery, hour(availability.time_queried) as hourquery, available_bikes,temp,wind_speed,pressure,humidity,weather_main FROM dublin_bikes.availability, dublin_bikes.weather_current
where availability.number = 2 && weather_current.station_number = 2 && timestampdiff(MINUTE,availability.time_queried, weather_current.time_queried) < 5 && timestampdiff(MINUTE,availability.time_queried, weather_current.time_queried) > 0
order by availability.time_queried;
""" 
# create select statement for stations table

split to 80 and 20/20 or 10/20

df_availability = pd.read_sql('availability', engine)
df_station = pd.read_sql('station', engine)
df_station = pd.read_sql('station', engine)
df_weather = pd.read_sql('hourly', engine)

In [ ]:
%load_ext sql
SHOW COLUMNS FROM CurrentWeather;
SHOW COLUMNS FROM ExtremeWeather;
SHOW COLUMNS FROM FiveDayPrediction;
SHOW COLUMNS FROM station;
SHOW COLUMNS FROM station_status;

In [ ]:
SELECT ss.station_number,
       ss.last_update,
       ss.status,
       ss.empty_stands_number,
       ss.total_bikes,
       ss.mechanical_bikes,
       ss.electrical_internal_battery_bikes,
       ss.electrical_removable_battery_bikes,
       s.name,
       s.address,
       s.banking,
       s.bonus,
       s.position_lat,
       s.position_lng,
       cw.feels_like,
       cw.temperature_min,
       cw.temperature_max,
       cw.weather_description,
       cw.wind_speed,
       cw.wind_gust,
       fdp.temp_min AS forecast_temp_min,
       fdp.temp_max AS forecast_temp_max,
       fdp.wind_speed AS forecast_wind_speed,
       fdp.gust AS forecast_gust,
       fdp.rain_3h AS forecast_rain_3h
FROM station_status ss
JOIN station s ON ss.station_number = s.number
JOIN currentweather cw ON ss.last_update = cw.time_update
JOIN FiveDayPrediction fdp ON ss.last_update = fdp.time_update;